In [1]:
from ngsolve import *
from ngsolve.meshes import Make1DMesh
from ngsolve.webgui import Draw
import numpy as np
import math

In [2]:
from sympy import symbols, simplify, solve, diff, lambdify
import sympy as sym
from sympy import Symbol
import scipy.integrate
import matplotlib.pyplot as plt
mesh = Make1DMesh(100)

X,T = symbols("x t")

u_x = X*(1-X)*sym.sin(4*sym.pi*X)*T**2*sym.exp(-T)

u_diff_x = diff(u_x, X)

u_ddiff_x = diff(u_diff_x,X)

u_dddiff_x = diff(u_ddiff_x,X)

u_ddddiff_x = diff(u_dddiff_x,X)

u_diff_t = diff(u_x, T)

u_ddiff_t = diff(u_diff_t,T)

f_x_t = u_ddddiff_x + u_ddiff_t

f = (T**2*X*(1-X) + 256*sym.pi**4*T**2*X*(1-X) + 192*sym.pi**2*T**2 - 4*T*X*(1-X) + 2*X*(1-X))*(sym.exp(-T)*sym.sin(4*sym.pi*X)) \
                      + (2*X - 1)*256*sym.pi**3*T**2*sym.exp(-T)*sym.cos(4*sym.pi*X)

In [3]:
u = H1(mesh, order=2, dirichlet="left|right")
sigma = H1(mesh, order=2, dirichlet=" ")
u_1 = NumberSpace(mesh)
fesm = u*sigma*u_1

In [4]:
u, sigma, u_1= fesm.TrialFunction()
v, tau, v_1 = fesm.TestFunction()

time = 0.0
dt = 0.5

In [5]:
a = BilinearForm(fesm, symmetric=False)
a += -grad(sigma)*grad(v)*dx + sigma*tau*dx + grad(u)*grad(tau)*dx -u_1*v_1*dx
a.Assemble()

m = BilinearForm(fesm, symmetric=False)
m += u_1*v*dx - u*v_1*dx
m.Assemble()

In [6]:
gfu = GridFunction(fesm)

In [7]:
class sdirk1: #order 1 (implicit Euler)
    stages = 1
    a = [[1]]
    b = [1]
    c = [1]
    astar = 1

In [8]:
class sdirk2: #order 3
    p = (3 - sqrt(3))/6
    stages = 2
    a = [[p, 0], 
         [1 - 2*p, p]]
    b = [1/2, 1/2]
    c = [p, 1 - p]
    astar = p

In [9]:
class sdirk5: #order 4
    stages = 5
    a = [[1/4, 0, 0, 0, 0], 
         [1/2, 1/4, 0, 0, 0], 
         [17/50,-1/25, 1/4, 0, 0],
         [371/1360, -137/2720, 15/544, 1/4,0],
         [25/24, -49/48, 125/16, -85/12, 1/4]]
    b = [25/24, -49/48, 125/16, -85/12, 1/4]
    c = [1/4, 3/4, 11/20, 1/2, 1]
    astar = 1/4 

In [10]:
butchertab = sdirk5()   
rhsi = gfu.vec.CreateVector()
Mu0 = gfu.vec.CreateVector()
ui = gfu.vec.CreateVector()
k = [gfu.vec.CreateVector() for i in range(butchertab.stages)]

In [11]:
mstar = m.mat.CreateMatrix()
mstar.AsVector().data = m.mat.AsVector() + butchertab.astar * dt * a.mat.AsVector()
invmstar = mstar.Inverse(freedofs=fesm.FreeDofs())
invmass = m.mat.Inverse(freedofs=fesm.FreeDofs())

NgException: UmfpackInverse: Numeric factorization failed.

In [ ]:
t = Parameter(0.0)

In [ ]:
omega=1
gausspt = exp(-6*((x+sin(omega*t))*(x+sin(omega*t))+y*y))-exp(-6*((x-sin(omega*t))*(x-sin(omega*t))+y*y))

In [ ]:
ft = LinearForm(fesm)
ft += gausspt*v*dx
# uD = ((1-y*y)*x)
time = 0.0
t.Set(0.0)
gfu.components[2].Set(x*(1-x)*sin(4*pi*x)*t**2*exp(-t))
# visualization stuff
from ngsolve.internal import *
visoptions.mminval = 0.0
visoptions.mmaxval = 0.2
visoptions.deformation = 0
visoptions.autoscale = 0


In [ ]:
mesh = Make1DMesh(80)
x=np.linspace(0,1,100)
gfu.components[0](mesh(x))

tstep = 10 # time that we want to step over within one block-run
t_intermediate=0 # time counter within one block-run
while t_intermediate < tstep - 0.5 * dt:
    ft.Assemble()
    Mu0.data = m.mat * gfu.vec + dt*ft.vec
    for i in range(butchertab.stages):
        # add up the ks as prescribed in the butcher tableau
        rhsi.data = Mu0
        for j in range(0,i):
            rhsi.data += dt * butchertab.a[i][j] * k[j]
        # Solve for ui (with homogenization for the boundary data)
        t.Set(time+t_intermediate+butchertab.c[i]*dt)
        gfu.components[2].Set(x*(1-x)*sin(4*pi*x)*t**2*exp(-t))
        ui.data = gfu.vec; rhsi.data -= mstar * ui
        ui.data += invmstar * rhsi
        # compute k[i] from ui
        k[i].data = - a.mat * ui
    t_intermediate += dt; t.Set(time+t_intermediate)
    # Adding up the ks:
    gfu.components[2].Set(x*(1-x)*sin(4*pi*x)*t**2*exp(-t))
    Mu0.data -= m.mat * gfu.vec
    for i in range(0,butchertab.stages):
        Mu0.data += dt * butchertab.b[i] * k[i]
    gfu.vec.data += invmass * Mu0 
    t_intermediate += dt
    plt.ylim([-0.2, 0.2])
    plt.xlim([0, 1])
    font1 = {'family':'serif','color':'black','size':15}
    plt.title("Numerical Solution",fontdict = font1)
    plt.xlabel('x',fontdict = font1)
    plt.ylabel('u(x,t)',fontdict = font1)
    plt.plot(x,gfu.components[0](mesh(x)),color='black',linestyle="-")
    plt.show()
    print("\r",time+t_intermediate,end="")
print("")
time+=t_intermediate